# 载入所需包

In [5]:
# coding: utf-8
import json
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from selenium import webdriver  
import time  

# 读取可用cookie值用于模拟登录

In [6]:
cookies={
    'w_uuid':'fwXhQBIWy_UntT9HbHpLdM7GKaoKJgUpowASTvek5zIDn-6mI83JxllNbLIyWp8k',
    'uuid':'51e901eb4dd745b8929a.1502260061.1.0.0',
    'lt':'2Ct_GRwGTcvLAO840fhPl4K2srkAAAAAdAQAAPlXlQNJssw00aC4BNOYtBANyu-AGIRlBkhYrfdlEf1pcQTps83yLL6W0x53yVjgOw',
    'n':'QWd368644487',
    'cookie_phone':'13381795391',
    '_ga':'GA1.2.1585922450.1502118010',
    '_gid':'GA1.2.400973617.1503672828',
    'w_cid':'310117',
    'w_cpy_cn':"%E6%9D%BE%E6%B1%9F%E5%8C%BA",
    'w_cpy':'songjiangqu',
    'waddrname':"%E6%9D%BE%E6%B1%9F%E5%A4%A7%E5%AD%A6%E5%9F%8E", 
    'w_geoid':'wtw0wy5rzxhd',
    'w_ah':"31.059717871248722,121.23892195522785,%E6%9D%BE%E6%B1%9F%E5%A4%A7%E5%AD%A6%E5%9F%8E|31.059717871248722,121.23892195522785,%E6%8D%B7%E5%AE%89%E7%89%B9%E4%B8%93%E5%8D%96%E5%BA%97%28%E5%A5%89%E8%B4%A4%E5%A4%A7%E5%AD%A6%E5%9F%8E%E4%B8%93%E5%8D%96%E5%BA%97%29|30.840984899550676,121.5308167040348,%E6%A0%BC%E6%9E%97%E8%B1%AA%E6%B3%B0%E9%85%92%E5%BA%97%28%E4%B8%8A%E6%B5%B7%E5%A5%89%E8%B4%A4%E5%A4%A7%E5%AD%A6%E5%9F%8E%E6%B5%B7%E6%80%9D%E8%B7%AF%E5%BA%97%29|30.852647982537746,121.52313686907291,IU%E9%85%92%E5%BA%97%28%E4%B8%8A%E6%B5%B7%E5%A5%89%E8%B4%A4%E6%B5%B7%E6%B9%BE%E5%A4%A7%E5%AD%A6%E5%9F%8E%E5%BA%97%29|31.05417190119624,121.78945876657963,%E5%8D%97%E6%B1%87%E5%A4%A7%E5%AD%A6%E5%9F%8E|30.88789489120245,121.92092690616846,%E6%98%93%E4%BD%B0%E8%BF%9E%E9%94%81%E6%97%85%E5%BA%97%E6%B5%A6%E4%B8%9C%E4%B8%B4%E6%B8%AF%E5%A4%A7%E5%AD%A6%E5%9F%8E%E5%BA%97|30.88733883574605,121.90849184989929,%E4%B8%B4%E6%B8%AF%E5%A4%A7%E5%AD%A6%E5%9F%8E%E5%85%AC%E4%BA%A4%E6%9E%A2%E7%BA%BD|30.882493928074837,121.9082598388195,%E4%B8%B4%E6%B8%AF%E5%A4%A7%E5%AD%A6%E5%9F%8E|31.29067299887538,121.5122977271676,%E5%90%8C%E6%B5%8E%E5%A4%A7%E5%AD%A6%E5%9F%8E-%E8%A7%84%E6%96%B0%E6%A5%BC|31.310605835169554,121.49721499532461,%E5%A4%A7%E5%AD%A6%E5%9F%8E%E5%85%AC%E5%AF%93",
    'JSESSIONID':'1c8zjmd6vj08r80b3hifelcrc',
    '__mta':'145184651.1502118013113.1503677753290.1503742008949.192',
    'w_visitid':'444e51de-2017-49a9-a675-400195e3723a',
    'w_utmz':"utm_campaign=baidu&utm_source=1522&utm_medium=(none)&utm_content=(none)&utm_term=(none)"
}

# 定义抓取商铺ID函数，包含模拟登录，模拟点击

In [7]:
def fetchrestaurant(url):
    driver = webdriver.Chrome()
    driver.get(url)
    driver.delete_all_cookies()    
    for cookie in cookies:
        driver.add_cookie({'name':cookie, 'value':cookies[cookie]})
    
    for i in range(0,10): 
        elem = driver.find_element_by_class_name("loading")
        elem.click()
        print("wait for 10 seconds for loading...")
        time.sleep(10)
    print("wait for 10 seconds for all source") # 等待网页传入
    time.sleep(10)
    print(driver.page_source) # 查看点击后网页
    
    ids=[]    
    for link in driver.find_elements_by_xpath("//*[@data-rid]"):
        ids.append(link.get_attribute('data-rid'))
        print (link.get_attribute('data-rid'))
    return ids

# 页面评论循环

In [8]:
def pageloop(pagemax):
    url_comment = 'http://waimai.meituan.com/ajax/comment'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'}
    formdata = {
        'wmpoiIdStr': '44928110682617800',# 不用改
        'offset':'1',
        'has_content':'1',
        'score_grade':'0',
        'uuid':'44brb8QmEQ3f2FJnVohE-Uhl_RyagPtPtasDpN9CMp4QCKHHAcCgKoygJJleFkCV',
        'platform':'1',
        'parner':'4',
        'originUrl':'http%3A%2F%2Fwaimai.meituan.com%2Fcomment%2F44928110682617800'# 同上
    }
    
    file = open(ip+id+'comments.txt','w',encoding='utf-8') # 根据地点ID和商铺ID命名
    commentlist=[]
    for id in ids:
        formdata['wmpoiIdStr']=id
        formdata['originUrl']='http%3A%2F%2Fwaimai.meituan.com%2Fcomment%2F{}'.format(id)
        print(formdata['wmpoiIdStr'],formdata['originUrl'])
        
        for i in range(1,pagemax):
            formdata['offset']=i
            urleach = requests.post(url=url_comment,data=formdata,headers=headers)
            # print(urleach.text)
            # 检查 ID 是否过期
            if urleach.text=='redirect:/404':
                print('None')
                break
            else:
                urleachjson = urleach.json()
                # 检查是否是最后一页
                if urleachjson['data']['wmCommentVo'] == []:
                    print('last page')
                    break
                else:
                    length = len(urleachjson["data"]["wmCommentVo"])
                    for l in range(0,length):
                        comment=urleachjson["data"]["wmCommentVo"][l]["wmComment"]["clean_comment"]
                        commentlist.append(comment)
    print("finish")
    file.write(str(commentlist))
    file.close()

# 开始抓取

In [ ]:
# place_id=['wtw0wy5rzxhd', 'wtw0wj99uqth', 'wtw0tvu7j91r', 'wtw6j51b218w', 'wtw6hezt6rcz', 'wtw6j0693u0r', 'wtqxwjfzs65h', 'wtqxwng5cc21', 'wtqxwqup58bq', 'wtw8emhrsj4x', 'wtqrt2d3wq6w', 'wtqrmw2jc231', 'wtqrmpj24ggh','wtqrt2d3wq6w', 'wtqrmw2jc231']
place_id=['wtw0wy5rzxhd']
for ip in place_id:
    print("wait for 10 second for next place")
    time.sleep(10)
    url='http://waimai.meituan.com/home/{}'.format(ip)
    print(url)
    fetchrestaurant(url)
    pageloop(1000)